In [3]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing

In [4]:
df = pd.read_csv('./data/raw_data_cleaned.csv')

In [5]:
df = df.rename(columns={"Unnamed: 0": "Food_name"})

In [6]:
df

,Food_name,Water (g),Energy (kal),Protein (g),lipid (g),Carbohydrate (g),Fiber (g),Sugars (g),Ash (g),Ca (mg),...,phenolics (mg),pH,Plain Occurences,Cool Occurences,Warm Occurences,Cold Occurences,Heavy Cold Occurences,Heavy Warm Occurences,Hot Occurences,Heavy Hot Occurences
0,Alfalfa,90.20,32.0,5.00,0.70,2.90,1.9,0.20,1.2,112.0,...,378.00,NaN,3,0,0,0,0,0,0,0
1,dolichos sinensis,90.10,18.0,2.20,0.30,7.30,4.3,NaN,0.7,62.0,...,39.20,NaN,3,0,0,0,0,0,0,0
2,hazelnuts,5.31,628.0,14.95,60.75,16.70,9.7,4.34,4.1,114.0,...,274.00,NaN,3,0,0,0,0,0,0,0
3,"Beans, kidney",11.75,333.0,23.58,0.83,60.01,24.9,2.23,0.9,143.0,...,27.40,5.70,3,0,0,0,0,0,0,0
4,"Peanuts,",6.50,567.0,25.80,49.24,16.13,8.5,4.72,3.2,92.0,...,356.70,7.50,3,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Lamb,59.47,282.0,16.56,23.41,0.00,0.0,0.00,1.0,16.0,...,0.00,6.05,0,0,1,0,0,0,0,0
237,"Nectarines, raw",87.59,44.0,1.06,0.32,10.55,1.7,7.89,NaN,6.0,...,533.00,4.05,0,0,1,0,0,0,0,0
238,"Onions, raw",89.11,40.0,1.10,0.10,9.34,1.7,4.24,0.5,23.0,...,84.50,5.46,0,0,1,0,0,0,0,0
239,Garlic sprouts,88.90,37.0,2.10,0.40,8.00,1.8,NaN,0.6,24.0,...,56.28,NaN,0,0,1,0,0,0,0,0


In [4]:
def minmax_normalize(df):
    X = df.drop(['Food_name'],axis=1)
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(X)
    X_array = scaler.transform(X)
    new_data = pd.DataFrame(X_array,columns = X.columns)
    new_data.insert(0, 'Food_name', df['Food_name'])
    return new_data

In [5]:
def occurrences(df):
    df["% Plain"] = df["Plain Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Cool"] = df["Cool Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Warm"] = df["Warm Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Cold"] = df["Cold Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Cold"] = df["Heavy Cold Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Warm"] = df["Heavy Warm Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Hot"] = df["Hot Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    df["% Heavy Hot"] = df["Heavy Hot Occurences"]/(df["Plain Occurences"] + df["Cool Occurences"] + df["Warm Occurences"] + df["Cold Occurences"] + df["Heavy Cold Occurences"] + df["Heavy Warm Occurences"] + df["Hot Occurences"] + df["Heavy Hot Occurences"])

    return df

In [6]:
def linear_scale(df):
    df["hot_cold_scale"] = (df["% Plain"] * (3.0/6.0)) + (df["% Cool"] * (2.0/6.0)) + (df["% Warm"] * (4.0/6.0)) + (df["% Cold"] * (1.0/6.0)) + (df["% Heavy Cold"] * (0.0/6.0)) + (df["% Heavy Warm"] * (4.5/6.0)) + (df["% Hot"] * (5.0/6.0)) + (df["% Heavy Hot"] * (6.0/6.0))

    return df

In [7]:
def mode(df):
    df['Plain'] = df['Plain Occurences'].astype(int)
    df['Cold'] = df['Cool Occurences'].astype(int) + df['Cold Occurences'].astype(int) + df['Heavy Cold Occurences'].astype(int)
    df['Hot'] = df['Warm Occurences'].astype(int) + df['Heavy Warm Occurences'].astype(int) + df['Hot Occurences'].astype(int) + df['Heavy Hot Occurences'].astype(int)
    newDf = pd.DataFrame(df.loc[:, ['Plain', 'Cold', 'Hot']].idxmax(axis=1))
    df["Mode"] = newDf[0]
    
    def label_mode(row):
        if row['Mode'] == 'Plain':
            return 1
        elif row['Mode'] == 'Cold':
            return 0
        elif row['Mode'] == 'Hot':
            return 2
        else:
            return 3

    df['Mode_code'] = df.apply (lambda row: label_mode(row), axis=1)
    
    return df

In [8]:
def iterative_imputation(df):
    ii_imp = IterativeImputer(estimator=ExtraTreesRegressor(), max_iter=10, random_state=1121218)

    name = df.loc[:, "Food_name"]
    inputs = df.loc[:, 'Water (g)':'pH']
    outputs = df.loc[:, 'pH':].drop(columns='pH')
    
    new_cols = inputs.columns[inputs.isnull().mean() < 0.3]
    inputs = inputs[new_cols]
    
    newinputs = ii_imp.fit_transform(inputs)
    newinputs = pd.DataFrame(newinputs, columns = new_cols) 


    return pd.concat([name, newinputs, outputs], axis=1)

In [9]:
def label_encoder(df):
    # reading word labels
    label_encoder = preprocessing.LabelEncoder()

    # encode word labels in column
    df['hot_cold_scale'] = label_encoder.fit_transform(df['hot_cold_scale'])

    df['hot_cold_scale'].unique()

    return df

In [10]:
def data_process(df):
    df = minmax_normalize(df)
    df = occurrences(df)
    df = linear_scale(df)
    df = mode(df)
    
    df = iterative_imputation(df)
  
    df = label_encoder(df)
    
    return df

In [11]:
df = data_process(df)
df

/Users/aaronxie/.local/share/virtualenvs/analysis-gniU7RWb/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,Food_name,Water (g),Energy (kal),Protein (g),lipid (g),Carbohydrate (g),Fiber (g),Ash (g),Ca (mg),Fe (mg),...,% Heavy Cold,% Heavy Warm,% Hot,% Heavy Hot,hot_cold_scale,Plain,Cold,Hot,Mode,Mode_code
0,Alfalfa,0.903084,0.036199,0.092251,0.0070,0.029565,0.035714,0.012024,0.093645,0.028747,...,0.0,0.0,0.0,0.0,13,1,0,0,Plain,1
1,dolichos sinensis,0.902082,0.020362,0.040590,0.0030,0.074421,0.080827,0.007014,0.051839,0.008214,...,0.0,0.0,0.0,0.0,13,1,0,0,Plain,1
2,hazelnuts,0.053164,0.710407,0.275830,0.6075,0.170252,0.182331,0.041082,0.095318,0.048255,...,0.0,0.0,0.0,0.0,13,1,0,0,Plain,1
3,"Beans, kidney",0.117641,0.376697,0.435055,0.0083,0.611785,0.468045,0.009018,0.119565,0.084189,...,0.0,0.0,0.0,0.0,13,1,0,0,Plain,1
4,"Peanuts,",0.065078,0.641403,0.476015,0.4924,0.164441,0.159774,0.032064,0.076923,0.047023,...,0.0,0.0,0.0,0.0,13,1,0,0,Plain,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Lamb,0.595414,0.319005,0.305535,0.2341,0.000000,0.000000,0.010020,0.013378,0.015914,...,0.0,0.0,0.0,0.0,16,0,0,0,Plain,1
237,"Nectarines, raw",0.876952,0.049774,0.019557,0.0032,0.107554,0.031955,0.002941,0.005017,0.002875,...,0.0,0.0,0.0,0.0,16,0,0,0,Plain,1
238,"Onions, raw",0.892171,0.045249,0.020295,0.0010,0.095219,0.031955,0.005010,0.019231,0.002156,...,0.0,0.0,0.0,0.0,16,0,0,0,Plain,1
239,Garlic sprouts,0.890068,0.041855,0.038745,0.0040,0.081558,0.033835,0.006012,0.020067,0.008214,...,0.0,0.0,0.0,0.0,16,0,0,0,Plain,1
